In [11]:
import pandas as pd

from src.data_processing.wrangling import cat_conditions, map_comp, map_investimento, map_importo

In [2]:
p01 = pd.read_parquet("data/updated/p01.parquet")
p02 = pd.read_parquet("data/updated/p02.parquet")
p03 = pd.read_parquet("data/updated/p03.parquet")
p04 = pd.read_parquet("data/updated/p04.parquet")
p05 = pd.read_parquet("data/updated/p05.parquet")

componenti = pd.read_parquet("data/updated/components.parquet")
missions = pd.read_parquet("data/updated/missions.parquet")
comuni = pd.read_parquet("data/updated/comuni.parquet")

indicatori_sdg = pd.read_parquet("data/updated/indicators_sdg.parquet")
indicators = pd.read_parquet("data/updated/indicators.parquet")
targets = pd.read_parquet("data/updated/targets.parquet")

## Data preprocessing

In [3]:
p01.shape

(4801727, 2)

In [4]:
p01.columns

Index(['CIG', 'CUP'], dtype='object')

In [5]:
p02.columns

Index(['CUP', 'CODICE_STATO', 'STATO', 'CODICE_AREA_GEOGRAFICA',
       'AREA_GEOGRAFICA', 'CODICE_REGIONE', 'REGIONE', 'CODICE_PROVINCIA',
       'SIGLA_PROVINCIA', 'PROVINCIA', 'CODICE_COMUNE', 'COMUNE'],
      dtype='object')

In [6]:
p02.shape

(11587625, 12)

In [7]:
p02["CODICE_COMPONENTE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:4])
p02["CODICE_MISSIONE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:2])
p02["COMPONENTE"] = p02['NOME_TEMATICA'].apply(map_comp)
p02["INVESTIMENTO"] = p02['NOME_TEMATICA'].apply(map_investimento)

missions.columns = ["MISSIONE"]
componenti.columns = ["COMPONENTE"]

p02 = p02.merge(missions,left_on="CODICE_MISSIONE", right_on=missions.index, how='inner')
p02 = p02.drop_duplicates()
p02 = p02.merge(componenti,left_on="CODICE_COMPONENTE", right_on=componenti.index, how='inner')
p02 = p02.drop_duplicates()

KeyError: 'NOME_TEMATICA'

In [ ]:
p03 = p03.rename(columns={'cig':'CIG'})
p03['CATEGORIA'] = p03.apply(cat_conditions,axis=1)

In [ ]:

p04 = p04.rename(columns={'cig':'CIG'})

In [ ]:
p05 = p05.rename(columns={'cig':'CIG'})
p05['CLASSE_IMPORTO'] = p05['importo_complessivo_gara'].apply(map_importo)

In [ ]:
leftnew_3 = p05.merge(p03, on='cig', how='left')
leftnew_3 = leftnew_3.drop_duplicates()